In [3]:
import numpy as np
import pandas as pd
from sklearn import *
import tensorflow as tf
import tensorflow.contrib.learn as skflow

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [4]:
col = [c for c in train.columns if c not in ['id','target']]

x1, x2, y1, y2 = model_selection.train_test_split(train[col], train['target'], test_size=0.25, random_state=99)
x1 = x1.values.astype(np.float32)
x2 = x2.values.astype(np.float32)
y1 = y1.values.astype(np.int)
y2 = y2.values.astype(np.int)
xtest = test[col].values.astype(np.float32)

In [36]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_tf(pred, y):
    return gini(y, pred) / gini(y, y)

In [25]:
col = [tf.feature_column.numeric_column('x', shape=x1.shape[1:])]
clf = skflow.DNNClassifier(feature_columns=col, hidden_units=[600, 1200, 600], n_classes=2)
clf.fit(input_fn=tf.estimator.inputs.numpy_input_fn(x={'x': x1}, y=y1, shuffle=False), steps=10000)
auc = clf.evaluate(input_fn=tf.estimator.inputs.numpy_input_fn(x={'x': x2}, y=y2, num_epochs=1, shuffle=False))['auc']
print('AUC: ', auc)

preds = clf.predict(input_fn=tf.estimator.inputs.numpy_input_fn(x={'x': x2}, num_epochs=1, shuffle=False))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_environment': 'local', '_model_dir': '/tmp/tmpy7f1dawf', '_save_checkpoints_steps': None, '_master': '', '_task_type': None, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_num_ps_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ed69d6748>, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 0, '_evaluation_master': '', '_session_config': None, '_is_chief': True}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create

In [26]:
prep = list(preds)

In [27]:
predicted_classes = [p["classes"] for p in prep]

IndexError: invalid index to scalar variable.

In [23]:
preds = [float(p["probabilities"][1]) for p in preds]
print('Gini: ', gini_tf(preds, y2))

AssertionError: 

In [ ]:
preds = clf.predict(input_fn=tf.estimator.inputs.numpy_input_fn(x={'x': xtest}, num_epochs=1, shuffle=False))
preds = [float(p["probabilities"][1]) for p in preds]
test['target'] = preds
test[['id','target']].to_csv('tf_submission.csv', index=False)

## with Keras

In [ ]:
import keras
import sklearn.model_selection
import numpy as np
import pandas as pd
import gc

Using TensorFlow backend.


In [4]:
import time
from sklearn.preprocessing import LabelEncoder
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


def target_encode(trn_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

gc.enable()

trn_df = pd.read_csv("data/train.csv")
sub_df = pd.read_csv("data/test.csv")

id_test = sub_df.id
target = trn_df["target"]
trn_df.drop("target", axis = 1 , inplace = True)

train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]
# add combinations
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]
start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    trn_df[name1] = trn_df[f1].apply(lambda x: str(x)) + "_" + trn_df[f2].apply(lambda x: str(x))
    sub_df[name1] = sub_df[f1].apply(lambda x: str(x)) + "_" + sub_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(trn_df[name1].values) + list(sub_df[name1].values))
    trn_df[name1] = lbl.transform(list(trn_df[name1].values))
    sub_df[name1] = lbl.transform(list(sub_df[name1].values))

    train_features.append(name1)
    
trn_df = trn_df[train_features]
sub_df = sub_df[train_features]

f_cats = [f for f in trn_df.columns if "_cat" in f]

for f in f_cats:
    trn_df[f + "_avg"], sub_df[f + "_avg"] = target_encode(trn_series=trn_df[f],
                                         tst_series=sub_df[f],
                                         target=target,
                                         min_samples_leaf=200,
                                         smoothing=10,
                                         noise_level=0)

trn_df.ps_car_15 = trn_df.ps_car_15**2
sub_df.ps_car_15 = sub_df.ps_car_15**2

current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.0

In [5]:
x_all = trn_df.values
y_all = keras.utils.np_utils.to_categorical(target.values)

In [6]:
# Split train/valid datasets
x_train, x_valid, y_train, y_valid = sklearn.model_selection.train_test_split(x_all, y_all, test_size=0.1, random_state=0)

# Define model
model = keras.models.Sequential()
model.add(keras.layers.normalization.BatchNormalization(input_shape=tuple([x_train.shape[1]]))) #Normalize Z before activation
model.add(keras.layers.core.Dense(128, activation='relu'))  #apply activation function to Z after normalizing
model.add(keras.layers.core.Dropout(rate=0.3)) # drop out 30% neurons of layer 1
model.add(keras.layers.normalization.BatchNormalization())
model.add(keras.layers.core.Dense(64, activation='relu'))
model.add(keras.layers.core.Dropout(rate=0.3)) # drop out 30% neurons of layer 2
model.add(keras.layers.normalization.BatchNormalization())
model.add(keras.layers.core.Dense(32, activation='relu'))
model.add(keras.layers.core.Dropout(rate=0.5)) # drop out 50% neurons of layer 3
model.add(keras.layers.core.Dense(2, activation='sigmoid')) #sigmoid for binary classification

# optimizer is Adadelta optimization algorithm
# loss function is categorical_crossentropy
model.compile(loss="categorical_crossentropy", optimizer="adadelta",metrics=["accuracy"]) 
print(model.summary())

# Use Early-Stopping
#Early stopping is basically stopping the training once your loss starts to increase 
#(or in other words validation accuracy starts to decrease)
#callback_early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=0, mode='auto')

# Train model
model.fit(x_train, y_train, batch_size=1024, epochs=200, verbose=1) #,callbacks=[callback_early_stopping])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 50)                200       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               6528      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 64)                256       
__________

535690/535690 [==============================] - 18s 34us/step - loss: 0.1518 - acc: 0.9636
Epoch 65/200
535690/535690 [==============================] - 18s 34us/step - loss: 0.1517 - acc: 0.9636
Epoch 66/200
535690/535690 [==============================] - 18s 34us/step - loss: 0.1517 - acc: 0.9636
Epoch 67/200
535690/535690 [==============================] - 18s 34us/step - loss: 0.1517 - acc: 0.9636
Epoch 68/200
535690/535690 [==============================] - 18s 34us/step - loss: 0.1517 - acc: 0.9636
Epoch 69/200
535690/535690 [==============================] - 18s 34us/step - loss: 0.1518 - acc: 0.9636
Epoch 70/200
535690/535690 [==============================] - 18s 34us/step - loss: 0.1517 - acc: 0.9636
Epoch 71/200
535690/535690 [==============================] - 18s 34us/step - loss: 0.1517 - acc: 0.9636
Epoch 72/200
535690/535690 [==============================] - 18s 34us/step - loss: 0.1517 - acc: 0.9636
Epoch 73/200
535690/535690 [==============================] - 18s 34

535690/535690 [==============================] - 18s 34us/step - loss: nan - acc: 0.9636
Epoch 144/200
535690/535690 [==============================] - 18s 34us/step - loss: nan - acc: 0.9636
Epoch 145/200
535690/535690 [==============================] - 18s 34us/step - loss: nan - acc: 0.9636
Epoch 146/200
535690/535690 [==============================] - 18s 34us/step - loss: nan - acc: 0.9636
Epoch 147/200
535690/535690 [==============================] - 18s 34us/step - loss: nan - acc: 0.9636
Epoch 148/200
535690/535690 [==============================] - 18s 34us/step - loss: nan - acc: 0.9636
Epoch 149/200
535690/535690 [==============================] - 18s 34us/step - loss: nan - acc: 0.9636
Epoch 150/200
535690/535690 [==============================] - 18s 34us/step - loss: nan - acc: 0.9636
Epoch 151/200
535690/535690 [==============================] - 18s 34us/step - loss: nan - acc: 0.9636
Epoch 152/200
535690/535690 [==============================] - 18s 34us/step - loss: na

In [7]:
# Predict test cross validation
y_test = model.predict(x_valid)

In [56]:
y_test

array([[ 0.94992745,  0.03313947],
       [ 0.907556  ,  0.03448642],
       [ 1.        ,  0.01953601],
       ..., 
       [ 0.98985159,  0.03169642],
       [ 0.98092467,  0.03235414],
       [ 1.        ,  0.02161432]], dtype=float32)

In [58]:
gini_tf(y_test[:,1], y_valid[:,1])

0.25286866840384936